# HPC with TensorFlow 2

In [1]:
import tensorflow as tf
import tensorflow_probability as tfp
import zfit
from zfit import z
import numpy as np
import numba

As mentioned, TensorFlow is basically Numpy. Let's check that out

In [2]:
rnd1 = tf.random.uniform(shape=(10,),  # notice the "shape" argument: it's more picky than Numpy
                         minval=0,
                         maxval=10)
rnd2 = tf.random.uniform(shape=(10,),
                         maxval=10)

In [3]:
rnd1

<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([9.542807 , 9.44273  , 6.7489004, 6.580929 , 9.47954  , 2.803868 ,
       5.496625 , 1.801542 , 4.1632595, 9.235947 ], dtype=float32)>

This is in fact a "numpy array wrapped" and can explicitly be converted to an array

In [4]:
rnd1.numpy()

array([9.542807 , 9.44273  , 6.7489004, 6.580929 , 9.47954  , 2.803868 ,
       5.496625 , 1.801542 , 4.1632595, 9.235947 ], dtype=float32)

Other operations act as we would expect it

In [5]:
rnd1 + 10

<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([19.542807 , 19.44273  , 16.748901 , 16.580929 , 19.47954  ,
       12.803868 , 15.496625 , 11.801542 , 14.1632595, 19.235947 ],
      dtype=float32)>

... and it converts itself (often) to Numpy when needed.

In [6]:
np.sqrt(rnd1)

array([3.0891433, 3.0729024, 2.5978646, 2.5653322, 3.078886 , 1.6744754,
       2.3444881, 1.3422153, 2.0404067, 3.0390701], dtype=float32)

We can slice it...

In [7]:
rnd1[1:3]

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([9.44273  , 6.7489004], dtype=float32)>

...expand it....

In [8]:
rnd1[None, :, None]

<tf.Tensor: shape=(1, 10, 1), dtype=float32, numpy=
array([[[9.542807 ],
        [9.44273  ],
        [6.7489004],
        [6.580929 ],
        [9.47954  ],
        [2.803868 ],
        [5.496625 ],
        [1.801542 ],
        [4.1632595],
        [9.235947 ]]], dtype=float32)>

...and broadcast with the known (maybe slightly stricter) rules

In [9]:
matrix1 = rnd1[None, :] * rnd1[:, None]

## Equivalent operations

Many operations that exist in Numpy also exist in TensorFlow, sometimes with a different name.

In [10]:
tf.sqrt(rnd1)

<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([3.0891435, 3.0729027, 2.5978646, 2.565332 , 3.078886 , 1.6744754,
       2.3444881, 1.3422154, 2.0404067, 3.0390701], dtype=float32)>

In [11]:
tf.reduce_sum(matrix1, axis=0)  # with the axis argument to specify over which to reduce

<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([623.10846, 616.57385, 440.6772 , 429.7093 , 618.9774 , 183.08179,
       358.90845, 117.63375, 271.84482, 603.0717 ], dtype=float32)>

## TensorFlow kernels

In general, TensorFlow is preciser compared to Numpy and does less automatic dtype casting and asks more explicit for shapes. For example, integers don't work in the logarithm. However, this error message illustrates very well the kernel dispatch system of TensorFlow, so lets do it!

In [12]:
try:
    tf.math.log(5)
except tf.errors.NotFoundError as error:
    print(error)

Could not find valid device for node.
Node:{{node Log}}
All kernels registered for op Log :
  device='XLA_GPU'; T in [DT_FLOAT, DT_DOUBLE, DT_COMPLEX64, DT_BFLOAT16, DT_COMPLEX128, DT_HALF]
  device='XLA_CPU'; T in [DT_FLOAT, DT_DOUBLE, DT_COMPLEX64, DT_BFLOAT16, DT_COMPLEX128, DT_HALF]
  device='XLA_CPU_JIT'; T in [DT_FLOAT, DT_DOUBLE, DT_COMPLEX64, DT_BFLOAT16, DT_COMPLEX128, DT_HALF]
  device='XLA_GPU_JIT'; T in [DT_FLOAT, DT_DOUBLE, DT_COMPLEX64, DT_BFLOAT16, DT_COMPLEX128, DT_HALF]
  device='CPU'; T in [DT_COMPLEX128]
  device='CPU'; T in [DT_COMPLEX64]
  device='CPU'; T in [DT_BFLOAT16]
  device='CPU'; T in [DT_DOUBLE]
  device='CPU'; T in [DT_HALF]
  device='CPU'; T in [DT_FLOAT]
  device='GPU'; T in [DT_DOUBLE]
  device='GPU'; T in [DT_HALF]
  device='GPU'; T in [DT_FLOAT]
 [Op:Log]


What we see here: it searches the registered kernels and does not find any that supports this operation. We find different classifications:
- GPU: normal GPU kernel
- CPU: normal CPU kernel
- XLA: [Accelerated Linear Algebra](https://www.tensorflow.org/xla) is a high-level compiler that can fuse operations, which would result in single calls to a kernel, to a single kernel.

## tf.function

We now want to see the JIT in action. Therefore, we use the example from the slides and start modifying it.

In [13]:
def add_log(x, y):
    print('running Python')
    tf.print("running TensorFlow")
    x_sq = tf.math.log(x)
    y_sq = tf.math.log(y)
    return x_sq + y_sq

As seen before, we can use it like Python. To make sure that we know when the actual Python is executed, we inserted a print and a `tf.print`, the latter is a TensorFlow operation and therefore expected to be called everytime we compute something.

In [14]:
add_log(4., 5.)

running Python
running TensorFlow


<tf.Tensor: shape=(), dtype=float32, numpy=2.9957323>

In [15]:
add_log(42., 52.)

running Python
running TensorFlow


<tf.Tensor: shape=(), dtype=float32, numpy=7.6889133>

As we see, both the Python and TensorFlow operation execute. Now we can do the same with a decorator. Note that so far we entered pure Python numbers, not Tensors. Since we ran in eager mode, this did not matter so far.

In [16]:
@tf.function(autograph=True)
def add_log_tf(x, y):
    print('running Python')
    tf.print("running TensorFlow")
    x_sq = tf.math.log(x)
    y_sq = tf.math.log(y)
    return x_sq + y_sq

In [17]:
add_log_tf(1., 2.)

running Python
running TensorFlow


<tf.Tensor: shape=(), dtype=float32, numpy=0.6931472>

In [18]:
add_log_tf(11., 21.)  # again with different numbers

running Python
running TensorFlow


<tf.Tensor: shape=(), dtype=float32, numpy=5.442418>

As we see, Python is still run: this happens because 11. is not equal to 1., TensorFlow does not convert those to Tensors. Lets use it in the right way, with Tensors

In [19]:
add_log_tf(tf.constant(1.), tf.constant(2.))  # first compilation

running Python
running TensorFlow


<tf.Tensor: shape=(), dtype=float32, numpy=0.6931472>

In [20]:
add_log_tf(tf.constant(11.), tf.constant(22.))

running TensorFlow


<tf.Tensor: shape=(), dtype=float32, numpy=5.488938>

Now only the TensorFlow operations get executed! Everything else became static. We can illustrate this more extremely here

In [21]:
@tf.function(autograph=True)
def add_rnd(x):
    print('running Python')
    tf.print("running TensorFlow")
    rnd_np = np.random.uniform()
    rnd_tf = tf.random.uniform(shape=())
    return x * rnd_np, x * rnd_tf

In [22]:
add_rnd(tf.constant(1.))

running Python
running TensorFlow


(<tf.Tensor: shape=(), dtype=float32, numpy=0.70940965>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.10721016>)

The first time, the numpy code was executed as well, no difference so far. However, running it a second time, only the TensorFlow parts can change

In [23]:
add_rnd(tf.constant(1.))

running TensorFlow


(<tf.Tensor: shape=(), dtype=float32, numpy=0.70940965>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.03636861>)

In [24]:
add_rnd(tf.constant(2.))

running TensorFlow


(<tf.Tensor: shape=(), dtype=float32, numpy=1.4188193>,
 <tf.Tensor: shape=(), dtype=float32, numpy=1.8568926>)

We see now clearly: TensorFlow executes the function but _only cares about the TensorFlow operations_, everything else is regarded as static. This can be a large pitfall! If we would execute this function _without_ the decorator, we would get a different result, since Numpy is also sampling a new random variable every time

## Variables

TensorFlow offers the possibility to have statefull objects inside a compiled graph (which e.g. is not possible with Numba). The most commonly used one is the `tf.Variable`. Technically, they are automatically captured on the function compilation and belong to it.

In [25]:
var1 = tf.Variable(1.)

In [26]:
@tf.function(autograph=True)
def scale_by_var(x):
    print('running Python')
    tf.print("running TensorFlow")
    return x * var1

In [27]:
scale_by_var(tf.constant(1.))

running Python
running TensorFlow


<tf.Tensor: shape=(), dtype=float32, numpy=1.0>

In [28]:
scale_by_var(tf.constant(2.))

running TensorFlow


<tf.Tensor: shape=(), dtype=float32, numpy=2.0>

In [29]:
var1.assign(42.)
scale_by_var(tf.constant(1.))

running TensorFlow


<tf.Tensor: shape=(), dtype=float32, numpy=42.0>

As we see, the output changed. This is of course especially useful in the context of model fitting libraries, be it likelihoods or neural networks.

In [30]:
def add_rnd(x):
    print('running Python')
    tf.print("running TensorFlow")
    rnd_np = np.random.uniform()
    rnd_tf = tf.random.uniform(shape=())
    return x * rnd_np, x * rnd_tf

In [31]:
add_rnd(tf.constant(1.))

running Python
running TensorFlow


(<tf.Tensor: shape=(), dtype=float32, numpy=0.6883487>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.43687785>)

In [32]:
add_rnd(tf.constant(2.))

running Python
running TensorFlow


(<tf.Tensor: shape=(), dtype=float32, numpy=1.371141>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.23517442>)

This means that we can use Numpy fully compatible in eager mode, but not when decorated.

In [33]:
def try_np_sqrt(x):
    return np.sqrt(x)

In [34]:
try_np_sqrt(tf.constant(5.))

2.236068

In [35]:
try_np_sqrt_tf = tf.function(try_np_sqrt, autograph=False)  # equivalent to decorator

In [36]:
try:
    try_np_sqrt_tf(tf.constant(5.))
except NotImplementedError as error:
    print(error)

Cannot convert a symbolic Tensor (x:0) to a numpy array.


As we see, Numpy complains in the graph mode, given that it cannot handle the Symbolic Tensor.

Having the `tf.function` decorator means that we can't use any Python dynamicity. What fails when decorated but works nicely if not:

In [37]:
def greater_python(x, y):
    if x > y:
        return True
    else:
        return False

In [38]:
greater_python(tf.constant(1.), tf.constant(2.))

False

This works again, and will fail with the graph decorator.

In [39]:
greater_python_tf = tf.function(greater_python, autograph=False)

In [40]:
try:
    greater_python_tf(tf.constant(1.), tf.constant(2.))
except Exception as error:
    print(error)

using a `tf.Tensor` as a Python `bool` is not allowed: AutoGraph is disabled in this function. Try decorating it directly with @tf.function.


The error message hints at something: while this does not work now - Python does not yet now the value of the Tensors so it can't decide whether it will evaluate to True or False - there is the possibility of "autograph": it automatically converts (a subset) of Python to TensorFlow: while loops, for loops through Tensors and conditionals. However, this is usually less effective and more errorprone than using explicitly the `tf.*` functions. Lets try it!

In [41]:
greater_python_tf_autograph = tf.function(greater_python, autograph=True)

In [42]:
greater_python_tf_autograph(tf.constant(1.), tf.constant(2.))

<tf.Tensor: shape=(), dtype=bool, numpy=False>

This now works neatless! But we're never sure.

## Performance

In the end, this is what matters. And a comparison would be nice. Let's do that and see how Numpy and TensorFlow compare.

In [43]:
nevents = 10000000
data_tf = tf.random.uniform(shape=(nevents,), dtype=tf.float64)
data_np = np.random.uniform(size=(nevents,))

In [44]:
def calc_np(x):
    x_init = x
    i = 42.
    x = np.sqrt(np.abs(x_init * (i + 1.)))
    x = np.cos(x - 0.3)
    x = np.power(x, i + 1)
    x = np.sinh(x + 0.4)
    x = x ** 2
    x = x / np.mean(x)
    x = np.abs(x)
    logx = np.log(x)
    x = np.mean(logx)
    
    x1 = np.sqrt(np.abs(x_init * (i + 1.)))
    x1 = np.cos(x1 - 0.3)
    x1 = np.power(x1, i + 1)
    x1 = np.sinh(x1 + 0.4)
    x1 = x1 ** 2
    x1 = x1 / np.mean(x1)
    x1 = np.abs(x1)
    logx = np.log(x1)
    x1 = np.mean(logx)
    
    x2 = np.sqrt(np.abs(x_init * (i + 1.)))
    x2 = np.cos(x2 - 0.3)
    x2 = np.power(x2, i + 1)
    x2 = np.sinh(x2 + 0.4)
    x2 = x2 ** 2
    x2 = x2 / np.mean(x2)
    x2 = np.abs(x2)
    logx = np.log(x2)
    x2 = np.mean(logx)
    return x + x1 + x2

calc_np_numba = numba.jit(nopython=True, parallel=True)(calc_np)

In [45]:
def calc_tf(x):
    x_init = x
    i = 42.
    x = tf.sqrt(tf.abs(x_init * (tf.cast(i, dtype=tf.float64) + 1.)))
    x = tf.cos(x - 0.3)
    x = tf.pow(x, tf.cast(i + 1, tf.float64))
    x = tf.sinh(x + 0.4)
    x = x ** 2
    x = x / tf.reduce_mean(x)
    x = tf.abs(x)
    x = tf.reduce_mean(tf.math.log(x))
    
    x1 = tf.sqrt(tf.abs(x_init * (tf.cast(i, dtype=tf.float64) + 1.)))
    x1 = tf.cos(x1 - 0.3)
    x1 = tf.pow(x1, tf.cast(i + 1, tf.float64))
    x1 = tf.sinh(x1 + 0.4)
    x1 = x1 ** 2
    x1 = x1 / tf.reduce_mean(x1)
    x1 = tf.abs(x1)
    
    x2 = tf.sqrt(tf.abs(x_init * (tf.cast(i, dtype=tf.float64) + 1.)))
    x2 = tf.cos(x2 - 0.3)
    x2 = tf.pow(x2, tf.cast(i + 1, tf.float64))
    x2 = tf.sinh(x2 + 0.4)
    x2 = x2 ** 2
    x2 = x2 / tf.reduce_mean(x2)
    x2 = tf.abs(x2)
    
    return x + x1 + x2

calc_tf_func = tf.function(calc_tf, autograph=False)

In [46]:
%%timeit -n1 -r1  # compile time, just for curiosity
calc_tf_func(data_tf)

219 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [47]:
%%timeit -n1 -r1  # compile time, just for curiosity
calc_np_numba(data_np)

2.9 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [48]:
%timeit calc_np(data_np)  # not compiled

2.36 s ± 66 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [49]:
%timeit calc_tf(data_tf)  # not compiled

721 ms ± 37.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [50]:
%%timeit -n1 -r7  # prevent caching
calc_np_numba(data_np)

427 ms ± 20.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [51]:
%%timeit -n1 -r7  # prevent caching
calc_tf_func(data_tf)

183 ms ± 10.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


We can now play around with this numbers. Depending on the size (we can go up to 10 mio) and parallelizability of the problem, the numbers differ..

In general:
- Numpy is faster for small numbers
- TensorFlow is faster for larger arrays and well parallelizable computations. Due to the larger overhead in dispatching in eager mode, it is significantly slower for very small (1-10) sample sizes.

=> there is no free lunch

Note: this has not run on a GPU, which would automatically happen for TensorFlow.

## Control flow

While TensorFlow is independent of the Python control flow, it has its own functions for that, mainly:
- while_loop(): a while loop taking a body and condition function
- cond(): if-like
- case and switch_case: if/elif statements
- tf.where

In [52]:
def true_fn():
    return 1.

def false_fn():
    return 0.

value = tf.cond(tf.greater(111., 42.), true_fn=true_fn, false_fn=false_fn)

In [53]:
value

1.0

## Gradients

TensorFlow allows us to calculate the automatic gradients.

In [54]:
var2 = tf.Variable(2.)

In [55]:
with tf.GradientTape() as tape:
    tape.watch(var2)  # actually watches all variables already by default
    y = var2 ** 3
y

<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

In [56]:
grad = tape.gradient(y, var2)
grad

<tf.Tensor: shape=(), dtype=float32, numpy=12.0>

This allows to do many things with gradients and e.g. solve differential equations.

## Statistics

While TensorFlow offers some support for statistical inference, TensorFlow-Probability is very strong at this and provides MCMC methods, probability distributions and more.

In [57]:
cauchy = tfp.distributions.Cauchy(loc=1., scale=10.)

In [58]:
sample = cauchy.sample(10)

In [59]:
cauchy.prob(sample)

<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([0.01792534, 0.00687814, 0.00084488, 0.01351866, 0.02930262,
       0.02684786, 0.00388126, 0.00253546, 0.00813279, 0.00580823],
      dtype=float32)>

### How they compare to zfit

TensorFlow-Probability offers a great choice of distributions to build a model. The flexibility in terms of vectorization and parametrization is larger than in zfit. However, they only provide analytic models and lack any numerical normalization or samplings.

Internally, zfit simply wraps the for certain implementations. There is also a standard wrapper, `WrapDistribution`, that allows to easily wrap any TFP distribution and use it in zfit.

# HowTo with zfit

Whenever possible, it is preferrable to write anything in TensorFlow. But given the possibility to mix, we can use this.
- try to use `z.py_function` or `tf.py_function` to wrap pure Python code
- if you write something and want to make sure it is run in eager mode, use `zfit.run.assert_executing_eagerly()`. This way, your function won't be compiled and an error would be raised.
- set the graph mode and numerical gradient accordingly

In [60]:
zfit.run.set_graph_mode(False)
zfit.run.set_autograd_mode(False)

In [61]:
class NumpyGauss(zfit.pdf.ZPDF):
    _PARAMS = ['mu', 'sigma']
    
    def _unnormalized_pdf(self, x):
        data = z.unstack_x(x)
        mu = self.params['mu']
        sigma = self.params['sigma']
        return tf.convert_to_tensor(np.exp( - 0.5 * (data - mu) ** 2 / sigma ** 2))

In [62]:
obs = zfit.Space('obs1', (-3, 3))
mu = zfit.Parameter('mu', 0., -1, 1)
sigma = zfit.Parameter('sigma', 1., 0.1, 10)


In [63]:
gauss_np = NumpyGauss(obs=obs, mu=mu, sigma=sigma)
gauss = zfit.pdf.Gauss(obs=obs, mu=mu, sigma=sigma)

In [64]:
integral_np = gauss_np.integrate((-1, 0))
integral = gauss.integrate((-1, 0))
print(integral_np, integral)

tf.Tensor([0.3422554], shape=(1,), dtype=float64) tf.Tensor([0.3422688], shape=(1,), dtype=float64)
